In [17]:
import os
import os.path as osp
import sys
import logging
import csv
import io
import base64
import ast

from tqdm import tqdm
import json


import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas  
from scipy.interpolate import griddata
from scipy.interpolate import RegularGridInterpolator



from flask import Flask, jsonify, request, render_template
from flask_cors import CORS 
from ragutils import ChatGpt, ragChat
from langchain_openai import ChatOpenAI
import importlib
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.documents import Document




In [3]:

modelDict = dict()
with open ('./shapetalk.csv', 'r', encoding='gbk') as f:
    counter = 0
    print('initializing')
    reader = csv.reader(f)
    for row in tqdm(reader):
        if (row[6] in modelDict):
            for i in range(1, 5):
                if (row[i] != ''):
                    modelDict[row[6]].add(row[i])
        else:
            modelDict[row[6]] = set()
        counter += 1

modelDict.pop('ShapeNet')
print (modelDict.keys())
embed_model = OpenAIEmbeddings(base_url="https://api3.apifans.com/v1")

vaseDocs = []
vaseId = []
i = 0
for sentence in list(modelDict['vase']):
    vaseDocs.append(Document(page_content=sentence))
    vaseId.append('vase' + str(i))
    i += 1



initializing


130343it [00:01, 123527.68it/s]


dict_keys(['source_object_class', 'faucet', 'cap', 'plant', 'cabinet', 'bookshelf', 'knife', 'table', 'skateboard', 'mug', 'vase', 'pistol', 'flowerpot', 'clock', 'chair', 'bathtub', 'bottle', 'display', 'bag', 'trashbin', 'scissors', 'person', 'helmet', 'bowl', 'airplane', 'guitar', 'dresser', 'bed', 'sofa', 'bench', 'lamp'])


In [4]:

vectorstore = Chroma.from_documents(documents=vaseDocs, embedding=embed_model, collection_name="model_discription_embed")

In [5]:
print (vectorstore)

In [7]:
query = "How can I change the vase?"
result = vectorstore.similarity_search(query ,k = 5)
result

[Document(metadata={}, page_content='The vase is a flower vase.'),
 Document(metadata={}, page_content='The vase has flowers in it'),
 Document(metadata={}, page_content='The vase is holding flowers'),
 Document(metadata={}, page_content='The top of the vase is open'),
 Document(metadata={}, page_content='The vase is a bowl.')]

In [8]:

ragClient = ragChat()

In [15]:
source_knowledge = 'The target\'s foot is more round and the vase is with builtiful color.'
modelClass = 'vase'
prompt_template = f"""以下内容为对一个{modelClass}类型物体的外型描述，请分析以下内容，并将内容按照格式以键值对的方式拆分，其中键为内容中对该物体提及的部位，如果是对整体外型的描述则直接用{modelClass}作为键，值为该部位对应的描述。请返回给我一个python键值对列表。不要使用代码格式返回:

内容:
{source_knowledge}

"""
ragClient.pushUserMessage(prompt_template)
answer = ragClient.getGptMsg()
print (answer)


[
    {"foot": "more round"}
]


In [19]:
parsedStr = json.loads(answer)
print (parsedStr)

[{'foot': 'more round'}]
